


<center>
<h3>  Tools </h3>
</center>
<center>
<h3> ToolKits</h3>
</center>
<center>
<h3>Agents</h3>
</center>




### Tools
Tools are utilities desgined to be called by model : their inputs are desgined to be generated by the model and their output are desgigned to be passed back to the model.
<i>Tools are needed when ever we want a model to control part of a code or call external API's</i>

A Tool consist of 
<ul>
<li>The <code>name</code> of the tool</li>
<li>The <code>description</code> of what the tool does</li>
<li>The <code>JSON schema </code> defining the input of the tool</li>
<li>The <code>function</code> (and optionally, an async variant of the function)</li>
</ul>

<strong>Two ways to call the tool</strong>
<ol>
<li>Invoke with just the arguments</li>
<li>Invoke with tool call.</li>
</ol>


Ways to create the tool
<ol>
<li>Using tool decorator</li>
<li>Using the Agents</li>
<li>By inheritaning <code>BaseTool</code> class</li>
</ol>



<hr>
<p><code>Best Practices</code></p>
When designing tools to be used by a model, it is important to keep in mind that :
<ul>
<li>Chat Model that have explicit <code>tool-calling API</code> will be better at tool calling then non fine tuned models.</li>
<li>Model will perform better if the tools have well choosen names, descriptions ans JSON schemas. This is another form of prompt engineering</li>
<li> Simply narrowly scoped tools are easier for models to use than complex tools</li>
</ul>



### ToolKits
<p>ToolKits are the collection of tools that are designed to be used together for a specific task. They have convenient loading methods</p>
<p>All toolkits expose a <code>get_tools</code> method which return a list of tools.</p>

<hr style="border: none; border-top: 3px dotted red"

In [19]:
"""
#initialize tool kit
tool_kit = ExampleToolkit(...)
#getting the list of tools
tools = tool_kit.get_tools()
"""

'\n#initialize tool kit\ntool_kit = ExampleToolkit(...)\n#getting the list of tools\ntools = tool_kit.get_tools()\n'

## Agents

<p>By themselves language model can't take actions- they just ouput text</p>
<p>Agents are system that use LLM as a reasoning engine to determine which action to take and what the input to those actions should be. The result of those action can be fed back into the agent and it determine whether more actions are needed or whether it is okay to finish.</p>

<p><code>LangGraph</code> is used to create highly controllable and cutomizable agents</p>

#### <CODE>Different ways to implementing Tool </CODE>

In [20]:
from langchain_community.llms.ollama import Ollama
# local model
llm = Ollama(model="llama3.1")  # To use it, run this in terminal. `ollama run llama3.1`

1. Sample Tool using decorator

In [21]:
from langchain_core.tools import tool,ToolException
@tool
def multiply_tool_decorator(a:int,b:int) -> int:
    """_summary_
    Multiply 2 numbers and return the value.Argumets are being type casted into int.

    Args:
        a (int): any number
        b (int): any number

    Returns:
        int: any number
    """
    if type(a) != int or type(b) != int:
        raise ToolException("Both the arguemnts must be fo integer type")
    return a*b
@tool
async def amultiply_tool_decor(a:int,b:int )-> int:
    """_summary_

    Args:
        a (int): int
        b (int): int

    Returns:
        int: int
    """
    return int(a*b)
# Tool initlization

print("Normal call(sync) ",multiply_tool_decorator.invoke({"a":"2","b":3}))
print("Async call ", await amultiply_tool_decor.ainvoke({"a":10,"b":20}))
# print("Normal call(sync) ",multiply_tool_decorator.invoke({"a":2,"b":"3"})) # No exception as being type casted in defined function.

Normal call(sync)  6
Async call  200


2. Sample Tool by subclassing <code>BaseTool</code>

In [22]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel,Field # To create type
from typing import Optional
from langchain_core.callbacks import AsyncCallbackManagerForToolRun

class CalculatorInput(BaseModel):
    a:int = Field(description="First number")
    b: int = Field(description="Second number")

class CustomCalculatorTool(BaseTool):
    name:str = "Sample Calculator"
    description:str= "For Simple mutliplication job"
    args_schema = CalculatorInput
    return_direct: bool = True
    
    def _run(self,a:int,b:int, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> int :
        """_summary_

        Args:
            a (int): int
            b (int): int
            run_manager (Optional[AsyncCallbackManagerForToolRun], optional): This Parameter is automatically
            being passed by Langchain during the tool execution. Defaults to None.

        Returns:
            int: _description_
        """
        return a*b
    
    async def _arun(self,a:int,b:int, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> int:
        return self._run(a,b,run_manager)
        


In [23]:
multiply = CustomCalculatorTool()
multiply

CustomCalculatorTool()

In [24]:
print("Normal call : ",multiply.invoke({"a":12,"b":56}))
print("Async call :",await multiply.ainvoke({"a": 2, "b": 3}))

Normal call :  672
Async call : 6
